In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.decomposition import PCA

In [ ]:
# Load the data
data = pd.read_csv('../resources/data_sdss.csv')
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
# keep only ra, dec, u, g, r, u, z, class and redshift columns
data_model = data[['ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'class', 'redshift']]
data_model.head()

In [ ]:
data_model.info()

In [ ]:
data_model.describe()

# Data Cleaning

In [ ]:
data_model['class'] = data_model['class'].map({'GALAXY':0, 'STAR':1})

clf = LocalOutlierFactor()
clf.fit_predict(data_model)
x_score = clf.negative_outlier_factor_
outlier_score = pd.DataFrame()
outlier_score["score"] = x_score

#filter for dropping                                           
filter2 = outlier_score["score"] < -1.5
outlier_index = outlier_score[filter2].index.tolist()
data_model = data_model.drop(outlier_index, inplace=False)
data_model['class'] = data_model['class'].map({0:'GALAXY', 1:'STAR'})

## Feature Engineering

In [ ]:
data_model['color_u_g'] = data_model['u'] - data_model['g']
data_model['color_g_r'] = data_model['g'] - data_model['r']
data_model['color_r_i'] = data_model['r'] - data_model['i']
data_model['color_i_z'] = data_model['i'] - data_model['z']

In [ ]:
data_model.describe()

# Data Visualization

In [ ]:
sns.set_context("paper", rc={"font.size":10,
                             "axes.titlesize":15,
                             "axes.labelsize":12,
                             "xtick.labelsize":10,
                             "ytick.labelsize":10,
                             "legend.fontsize":15})

palette = {'GALAXY':'#4daf4a',
           'STAR':'#ff7f00'}

In [ ]:
labels = [data_model['class'].value_counts().iloc[0],
          data_model['class'].value_counts().iloc[1]]

plt.figure(figsize=(10, 6))
data_model['class'].value_counts().plot(kind='barh', title='Comparison of Sky Objects',
                                        color=['#4daf4a', '#ff7f00', '#377eb8']).invert_yaxis()
plt.xlabel('Number of Observations')
plt.xlim(0, 400000)

for index, value in enumerate(labels):
    plt.text(value, index, str(value))

plt.show()

In [ ]:
f, axs = plt.subplots(1,2,
                      figsize=(15,8),
                      sharey=True,
                     gridspec_kw=dict(width_ratios=[3,0.8]))
sns.scatterplot(x = 'ra',y = 'dec', hue = 'class', data = data_model, ax = axs[0], palette = palette, alpha = 0.5)
sns.kdeplot(y = 'dec', hue = 'class', data = data_model, ax = axs[1], palette = palette, legend = False)
f.tight_layout()

plt.suptitle('Equatorial Coordinates', fontsize = 15);

In [ ]:
def get_hists(feature_name):
    if feature_name == 'redshift':
        fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(15, 5))
    else:
        fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(15, 5), sharex = True)
    
    ax = sns.histplot(data_model[data_model['class']=='GALAXY'][feature_name], bins = 30, ax = axes[0], 
                      color = '#4daf4a', kde = False)
    ax.set_title('Galaxy')
    ax = sns.histplot(data_model[data_model['class']=='STAR'][feature_name], bins = 30, ax = axes[1], 
                      color = '#ff7f00', kde = False)
    ax.set_title('Star')
    fig.suptitle(feature_name.upper(), fontsize = 15)
    fig.tight_layout(pad = 0.5)

In [ ]:
columns = list(data_model.drop(['class'], axis = 1).columns)
for name in columns:
    get_hists(name)

# Correlation matrix

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(15, 6))
fig.set_dpi(250)
ax = sns.heatmap(data_model[data_model['class']=='GALAXY'][
    ['u', 'g', 'r', 'i', 'z']].corr(), ax = axes[0], cmap='Spectral', annot=True)
ax.set_title('Galaxy')
ax = sns.heatmap(data_model[data_model['class']=='STAR'][
    ['u', 'g', 'r', 'i', 'z']].corr(), ax = axes[1], cmap='Spectral', annot=True)
ax.set_title('Star')
fig.tight_layout(pad = 0.5);

### Pair Plot

In [ ]:
sns.pairplot(data_model.sample(10000), hue = 'class', height = 4, palette = palette);

# Logistic Regression Model

In [ ]:
data_model.head()

In [ ]:
X = data_model.drop(['class'], axis = 1)
y = data_model['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2023, stratify=y)

In [ ]:
std_scale = StandardScaler()

X_train_scaled = std_scale.fit_transform(X_train)
X_test_scaled = std_scale.transform(X_test)

In [ ]:
def plot_confusion(model_name, model, x_values, actual_values):
    labels = ['GALAXY', 'STAR']
    # Predict the values using the model
    predicted_values = model.predict(x_values)
    
    # Compute the confusion matrix
    confusion = confusion_matrix(actual_values, predicted_values, labels=labels)
    
    # Plot the confusion matrix
    plt.figure(dpi=150)
    sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, fmt='g',
                square=True, xticklabels=labels, yticklabels=labels)

    plt.xlabel('Predicted class')
    plt.ylabel('Actual class')
    plt.title(f'{model_name} Confusion Matrix', fontsize=12)
    plt.show()


In [ ]:
linreg = LogisticRegression()

logreg_baseline = LogisticRegression(solver='lbfgs')

# Setup repeated stratified k-fold cross-validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# Evaluate the model using cross-validation
n_scores = cross_val_score(logreg_baseline, X_train_scaled, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

logreg_baseline.fit(X_train_scaled, y_train)

# Evaluate the model on the test set
test_accuracy = logreg_baseline.score(X_test_scaled, y_test)
print('Test Accuracy: %.3f' % test_accuracy)

In [ ]:
plot_confusion('Logistic Multinomial - Test Set',logreg_baseline, X_test_scaled, y_test)

In [ ]:
y_pred = logreg_baseline.predict(X_test_scaled)
print('Logistic Regression accuracy:', metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred, digits = 5))

# Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

y_pred = dtree.predict(X_test)

print('Simple Decision Tree')
print('Simple Decision Tree accuracy:', metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred, digits = 5))

In [ ]:
dtree.get_params()

In [ ]:
param_grid = {
    "max_depth" : [5,10,15],
    "min_samples_split": [2,4,6,8],
    "min_samples_leaf": [3,4,5,6],
    "max_features": ['auto', None],
}

grid = GridSearchCV(dtree, param_grid, cv=10, scoring='f1_weighted', verbose = 2)
grid.fit(X_train, y_train)

print("Best params: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

In [ ]:
best_dtree = DecisionTreeClassifier(max_depth=15, 
                                    min_samples_leaf=5, 
                                    min_samples_split = 6, 
                                    max_features = None)

best_dtree.fit(X_train, y_train)

In [ ]:
print('Decision Tree - Test Set')
y_pred = best_dtree.predict(X_test)
print('Decision Tree accuracy best params:', metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred, digits = 5))

In [ ]:
plot_confusion('Best Decision Tree - Test Set', best_dtree,  X_test, y_test)

# KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

# fit the knn parameters to training data set
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)
print('KNN accuracy - test set:', metrics.accuracy_score(y_test, y_pred))

In [ ]:
param_grid = {
    "n_neighbors": np.arange(1,12),
    "weights": ['uniform', 'distance'],
    "metric": ["euclidean","manhattan"],
    "n_jobs": [4]}
    
grid = GridSearchCV(knn, param_grid, cv=10, scoring='f1_weighted', verbose = 2)
grid.fit(X_train_scaled, y_train)

print("Best params: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors=6, weights='uniform', metric='manhattan', n_jobs=4)
best_knn.fit(X_train_scaled, y_train)

In [ ]:
print('KNN - Test Set')
y_pred = best_knn.predict(X_test_scaled)
print('KNN accuracy with best params:', metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred, digits = 5))

In [ ]:
plot_confusion("Best KNN - Test set", best_knn, X_test_scaled, y_test)

### Grid search - k neighbors

In [ ]:
k_range = list(range(1, 30))
param_grid = dict(n_neighbors=k_range)    
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', verbose = 2)
grid.fit(X_train_scaled, y_train)

In [ ]:
results = pd.DataFrame(grid.cv_results_)

In [ ]:
mean_test_scores = results['mean_test_score']
neighbors = results['param_n_neighbors']

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(neighbors, mean_test_scores, marker='o')
plt.title('Accuracy vs. Number of Neighbors (k)')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gauss_nb = GaussianNB()
gauss_nb.fit(X_train,y_train)

y_pred = gauss_nb.predict(X_test)

print('Gaussian Naive Bayes')
print('Gaussian Naive Bayes accuracy:', metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred, digits = 3))

In [ ]:
gauss_nb = GaussianNB()
param_grid = {
             "var_smoothing": np.logspace(-15,1,base=10,num=50)
             }

grid = GridSearchCV(gauss_nb, param_grid, cv = 10, scoring = 'f1_weighted', verbose = 2)
grid.fit(X_train, y_train)

print("Best params: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

In [ ]:
best_gauss = GaussianNB(var_smoothing=8.685113737513521e-13)

best_gauss.fit(X_train, y_train)
y_pred = best_gauss.predict(X_test)

print('Gaussian NB - Test Set')
print('Gaussian NB accuracy best params:', metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred, digits = 5))

In [ ]:
plot_confusion("Gaussian Naive Bayes", best_gauss, X_test, y_test)

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators = 100, random_state=42)

rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

print('Random Forest')
print('Random Forest accuracy:', metrics.accuracy_score(y_test, y_pred))


In [ ]:
rf.get_params()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
hyperparameters = {'max_features':[None, 'auto', 'sqrt', 'log2'],
                   'max_depth':[None, 1, 5, 10, 15, 20],
                   'min_samples_leaf': [1, 2, 4],
                   'min_samples_split': [2, 5, 10],
                   'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)],
                   'criterion': ['gini', 'entropy']}
rf_random = RandomizedSearchCV(RandomForestClassifier(), param_distributions=hyperparameters, scoring='accuracy', n_iter = 100, cv = 10, verbose=2, random_state=123, n_jobs = -1)
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
best_rf = rf_random.best_estimator_
y_pred = best_rf.predict(X_test)

In [ ]:
print('Random Forest - Test Set')
print('Random Forest accuracy with best params:', metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred, digits = 5))

In [ ]:
plot_confusion("Random Forest", best_rf, X_test, y_test)

### Feature importance

In [ ]:
importances_rf = pd.DataFrame({'feature': X_train.columns,
                             'importance': best_rf.feature_importances_})
importances_rf = importances_rf.sort_values('importance',ascending=False).set_index('feature')
importances_rf.head(17)